## 仅供交叉验证 线性回归-按总量分类

用岭回归效果好一些

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

np.random.seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


## 规范数据并获取5折交叉检验所需的训练集和验证集

In [6]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按总量切分数据

In [2]:
# def split_data_by_es(data, targets):
#     total_es = np.sum(data[:, :11, -2], axis=1)
#     q75, q25 = np.percentile(total_es, [75 ,25])
#     iqr = q75 - q25
#     upper_bond = q75 + 1.5 * iqr
#     lower_bond = q25 - 1.5 * iqr
    
#     data1, target1 = data[np.argwhere(total_es > upper_bond).reshape(-1)], targets[np.argwhere(total_es > upper_bond).reshape(-1)]
#     data2, target2 = data[np.argwhere((q75 < total_es) & (total_es <= upper_bond)).reshape(-1)], targets[np.argwhere((q75 < total_es) & (total_es <= upper_bond)).reshape(-1)]
#     data3, target3 = data[np.argwhere((q25 < total_es) & (total_es <= q75)).reshape(-1)], targets[np.argwhere((q25 < total_es) & (total_es <= q75)).reshape(-1)]
#     data4, target4 = data[np.argwhere(total_es <= q25).reshape(-1)], targets[np.argwhere(total_es <= q25).reshape(-1)]
    
#     return data1, target1, data2, target2, data3, target3, data4, target4

def split_data_by_es(data, targets):
    total_es = np.sum(data[:, :11, -2], axis=1)
    sorted_index = np.argsort(total_es)
    group_size = len(total_es) // 3
    
    data1, target1 = data[sorted_index[:group_size]], targets[sorted_index[:group_size]]
    data2, target2 = data[sorted_index[group_size:2*group_size]], targets[sorted_index[group_size:2*group_size]]
    data3, target3 = data[sorted_index[2*group_size:]], targets[sorted_index[2*group_size:]]
    
    return data1, target1, data2, target2, data3, target3

### 构建模型，训练并评估

In [3]:
def cross_validation(X, y, y_cat, kfold, scaler):

    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2, X_train3, y_train3 = split_data_by_es(X_train, y_train)
        train_xs = [X_train1, X_train2, X_train3]
        train_ys = [y_train1, y_train2, y_train3]
        
        X_test1, y_test1, X_test2, y_test2, X_test3, y_test3 = split_data_by_es(X_test, y_test)
        test_xs = [X_test1, X_test2, X_test3]
        test_ys = [y_test1, y_test2, y_test3]
        i_s = [1, 2, 3]
        
        # 训练
        for i in range(len(i_s)):
            model = RidgeCV(alphas=[0.1, 0.2, 0.3 ,0.5, 0.7, 1.0])
            model.fit(train_xs[i].reshape(len(train_xs[i]), -1), train_ys[i])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i].reshape(len(test_xs[i]), -1)).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [4]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('lr_metrics.dict', 'wb'))
    
    return metrics

In [5]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


In [7]:
metrics

{'gene': {'overall': {'mae': 0.803805069268714,
   'rmse': 1.3775809587350656,
   'ndcg': 0.27703130907410056,
   'mape': 6.241592991605529,
   'r2': 0.12426727691632804,
   'pearson': 0.4624543586048224,
   'acc': 0.3152363064933212},
  'annual': {'mae': array([0.46279295, 0.6275107 , 0.82585445, 0.96106809, 1.14179915]),
   'rmse': array([0.92048705, 1.1155934 , 1.42226793, 1.53168087, 1.71793626]),
   'ndcg': array([0.45859406, 0.25489129, 0.20299   , 0.11015942, 0.12125649]),
   'mape': array([4.59617418, 4.39850159, 5.13226385, 8.29537946, 8.78564586]),
   'r2': array([ 0.02778768, -0.01549241,  0.04519124, -0.01822958, -0.04018483]),
   'pearson': array([0.62824389, 0.45777221, 0.40923633, 0.28512082, 0.16624103]),
   'acc': array([0.55546241, 0.28486526, 0.31402509, 0.24852856, 0.17330022])}},
 'transplant': {'overall': {'mae': 0.8275970961134513,
   'rmse': 1.3227238534044254,
   'ndcg': 0.37346640852859464,
   'mape': 3.9552484394916156,
   'r2': 0.375207485520504,
   'pearson